In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.5 MB/s eta 0:00:00


In [2]:
!pip install wandb

In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "newT5"
os.environ["WANDB_NOTES"] = "Fine tune t5 without optimization"
os.environ["WANDB_NAME"] = "araT5_Base"

In [4]:
from datasets import load_dataset, DatasetDict

# Load the dataset with the first 40,000 examples
ds = load_dataset("Helsinki-NLP/un_pc", "ar-en", split="train[:20000]")

# Split the dataset into 20,000 for training, and 20,000 for test + validation
train_test_val_split = ds.train_test_split(test_size=5000)  # Keep 20,000 for test+validation

# Further split the 20,000 examples into 10,000 for test and 10,000 for validation
test_validation_split = train_test_val_split['test'].train_test_split(test_size=0.5)

# Create a new dataset dictionary with the updated splits
ds_split = DatasetDict({
    'train': train_test_val_split['train'],  # 20,000 examples for training
    'test': test_validation_split['train'],  # 10,000 examples for testing
    'validation': test_validation_split['test'],  # 10,000 examples for validation
})

# Check the resulting dataset
ds_split

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20044478 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2500
    })
})

In [5]:
print(ds_split)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 15000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2500
    })
})


# Prerpocessing

In [6]:
from transformers import AutoTokenizer

model_name="UBC-NLP/AraT5v2-base-1024"
tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from transformers import AutoModelForSeq2SeqLM
import torch
model=AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map={'':torch.cuda.current_device()})

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [9]:
source_lang="ar"
target_lang="en"
prefix="translate Arabic to English: "

def preprocess_function(examples):
    inputs =[prefix+example[source_lang] for example in examples["translation"]]
    targets=[example[target_lang] for example in examples["translation"]]
    model_inputs=tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs


tokenized_ds=ds_split.map(preprocess_function, batched=True)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
print(data_collator)

DataCollatorForSeq2Seq(tokenizer=T5TokenizerFast(name_or_path='UBC-NLP/AraT5v2-base-1024', vocab_size=110100, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_

# Evaluate



In [11]:
!pip install evaluate       
!pip install rouge_score
!pip install sacrebleu

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d995452155f8062fb4af7880c370173a3b1c92d09e761a99b75ee2eab6f493c4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
import evaluate
import numpy as np
import sacrebleu

metric_bleu = evaluate.load("sacrebleu")
metric_rouge = evaluate.load("rouge")

# Define a function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# Define a function to compute metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    # Compute BLEU score
    result_bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result_bleu = {"bleu": result_bleu["score"]}
#     references=decoded_labels
#     result_bleu = sacrebleu.corpus_bleu(translations=decoded_preds,[references])  
#     result_bleu = {"bleu": round(bleu.score, 2)}
    
    # Compute ROUGE score
    result_rouge = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result_rouge = {"rouge": result_rouge["rougeL"]}
    
    # Combine BLEU and ROUGE results
    result = {**result_bleu, **result_rouge}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    
    # Save the result to a file after each evaluation
    with open('metrics.txt', 'a') as f:
        f.write(f"BLEU: {result['bleu']}, ROUGE: {result['rouge']}, Gen Len: {result['gen_len']}\n")
    
    return result


# Training

In [13]:
print(model.config)

T5Config {
  "_name_or_path": "UBC-NLP/AraT5v2-base-1024",
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.44.0",
  "use_cache": true,
  "vocab_size": 110208
}



In [14]:
model

T5ForConditionalGeneration(
  (shared): Embedding(110208, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(110208, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, Trainer, get_linear_schedule_with_warmup
import gc
import os

training_args=Seq2SeqTrainingArguments(
    
    output_dir=os.getenv("WANDB_NAME"),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir=os.getenv("WANDB_NAME")+"/logs",
    logging_strategy="epoch",
    logging_steps=500,
    load_best_model_at_end=True,
    learning_rate=2e-4,
    per_device_train_batch_size= 2,
    per_device_eval_batch_size= 2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs= 7,
    predict_with_generate=True,
#     fp16=True,
#     push_to_hub=True,
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
)

# Create Trainer instance
trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
model.config.use_cache=False
torch.cuda.empty_cache()
gc.collect()
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: moudjaramina2001 (FinalProject_). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Bleu,Rouge,Gen Len
1,2.736500,1.682160,16.720200,0.567900,14.121600
2,1.633300,1.421314,19.291500,0.592400,14.716400
3,1.204600,1.335618,20.513500,0.606500,14.620400
4,0.924600,1.299340,21.057400,0.627900,14.329600
5,0.704300,1.309649,21.686600,0.626800,14.554000
6,0.546400,1.390451,21.968400,0.631100,14.521600
7,0.433100,1.446959,22.002400,0.631900,14.514800


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=52500, training_loss=1.1689760649181549, metrics={'train_runtime': 12518.4133, 'train_samples_per_second': 8.388, 'train_steps_per_second': 4.194, 'total_flos': 8032701737705472.0, 'train_loss': 1.1689760649181549, 'epoch': 7.0})

In [16]:
import math 
eval_results=trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Perplexity: 3.67


# Inference

In [17]:
from transformers import pipeline

# text=" المملكة العربية السعودية"
# translator=pipeline("translation",model=os.getenv("WANDB_NAME"))
# translator(text)
context=tokenizer(["إن مشكلة الحضارة ليست مشكلة علمية بحتة، ولا هي مشكلة تقنية يمكن حلها بوسائل مادية فقط، وإنما هي قبل كل شيء مشكلة إنسانية وأخلاقية. فهي تتعلق بالفرد الذي يصنع الحضارة، وليس بالأدوات التي يستخدمها في صناعتها"], return_tensors="pt")
output=ia3_model.generate(**context)

tokenizer.decode(output[0], skip_special_tokens=True)

NameError: name 'ia3_model' is not defined

In [ ]:
tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(os.getenv("WANDB_NAME"))